### Chapter 13
**CH11 Used cars with linear regression**

using the used-cars dataset

version 1.0 2023-12-28

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

### Get Data

In [ ]:
# Current script and repository folder
current_path = os.getcwd()
repository_path = current_path.split('Ch13-a-framework-for-prediction')[0]

In [ ]:
# Add utils folder to sys path 
# Note: os.path.join() creates a string with the right syntax for defining a path for your operating sytem.
sys.path.append(os.path.join(repository_path, 'utils'))

In [ ]:
# Define data folder
data_path = os.path.join(repository_path, 'data')

In [ ]:
# Import the prewritten helper functions
from py_helper_functions import *

In [ ]:
# DATA IMPORT - FROM GITHUB
data = pd.read_csv('https://raw.githubusercontent.com/peterduronelly/DA3-Python-Codes/main/data/used-cars_2cities_prep.csv')

In [ ]:
data.head()

In [ ]:
data.info()

### EDA

In [ ]:
# SAMPLE DESIGN

# Manage missing
data["fuel"] = data["fuel"].fillna("Missing")
data["condition"] = data["condition"].fillna("Missing")
data["drive"] = data["drive"].fillna("Missing")
data["cylinders"] = data["cylinders"].fillna("Missing")
data["transmission"] = data["transmission"].fillna("Missing")
data["type"] = data["type"].fillna("Missing")

In [ ]:
# drop hybrid models then drop column
data = data[data.Hybrid == 0].drop(
    ["Hybrid"], axis=1
) 

In [ ]:
data.shape

In [ ]:
# check frequency by fuel type
freq = data.groupby("fuel").agg(frequency=("type", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

In [ ]:
# keep gas-fuelled vehicles
data = data[data.fuel == "gas"]

In [ ]:
# check frequency by vehicle condition
freq = data.groupby("condition").agg(frequency=("type", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

In [ ]:
# drop vehicles in fair and new condition, trucks
data = data[~data.condition.isin(["new", "fair"])]

# drop unrealistic values for price and odometer reading
data = data[(data.price >= 500) & (data.price <= 25000) & (data.odometer <= 100)]

# drop if price is smaller than 1000 and condition is like new or age is less than 8
data = data[
    ~((data.price < 1000) & ((data.condition == "like new") | (data.age < 8)))
]

In [ ]:
# check frequency by transmission
freq = data.groupby("transmission").agg(frequency=("type", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq

In [ ]:
data = data[~(data.transmission == "manual")]

In [ ]:
# check frequency by transmission
freq = data.groupby("type").agg(frequency=("type", "size"))
freq["percent"] = round(freq["frequency"] / sum(freq["frequency"]) * 100, 3)
freq["cumulative_percent"] = np.cumsum(freq["percent"])
freq


In [ ]:
# drop if truck
data = data[~(data.type == "truck")]
# drop pricestr
data = data.drop(["pricestr"], axis=1)

### Feature Engineering

In [ ]:
# condition
data["cond_excellent"] = np.where(data["condition"] == "excellent", 1, 0)
data["cond_good"] = np.where(data["condition"] == "good", 1, 0)
data["cond_likenew"] = np.where(data["condition"] == "like new", 1, 0)

In [ ]:
# cylinders
data["cylind6"] = np.where(data["cylinders"] == "6 cylinders", 1, 0)

In [ ]:
data.cylinders.value_counts()

In [ ]:
data.cylind6.value_counts()

In [ ]:
# age: quadratic, cubic
data["agesq"] = data["age"] ** 2
data["agecu"] = data["age"] ** 3

In [ ]:
# odometer quadratic
data["odometersq"] = data["odometer"] ** 2

### Frequency tables

In [ ]:
# area
data.groupby("area").agg(frequency=("price", 'size'), mean=("price", np.mean))

Another way to calculate multiple aggregations:

In [ ]:
# area
data.groupby("area").agg({'price': ['count', 'mean']})

In [ ]:
# focus only on Chicago
data = data[data.area == "chicago"]

In [ ]:
# condition
data.groupby("condition").agg(frequency=("price", "size"), mean=("price", np.mean))

In [ ]:
# drive
data.groupby("drive").agg(frequency=("price", "size"), mean=("price", np.mean))

In [ ]:
# dealer
data.groupby("dealer").agg(frequency=("price", "size"), mean=("price", np.mean))

In [ ]:
# data summary
data[[
    "age",
    "odometer",
    "LE",
    "XLE",
    "SE",
    "cond_likenew",
    "cond_excellent",
    "cond_good",
    "cylind6",
    ]].describe().T

### Charts

We are using multiple ways to plot certain charts in this notebook. Python's primary plotting library is `matplotlib`(https://matplotlib.org/), which is very straightforward to start with but can easily be overwhelming when it comes to intricacies. A good intro can be found [here](https://fritz.ai/introduction-to-matplotlib-data-visualization-in-python/). 

There are multiple other plotting tools and libraries, most of which are some sort of wrapper around `matplotlib`. `seaborn` is a library for [analytical and statistical graphics](https://seaborn.pydata.org/tutorial/introduction.html), but sometimes it is sufficient to use `Pandas` `plot()` method for quick and simple charts.

`R`, one of the data science & analytics alternatives for Python, is famous for its `ggplot` package which has been implemented in Python under the name `plotnine`. The syntaxes for building plotnine charts are almost the same as in `ggplot`. 

In [ ]:
# For certain charts, we need to sort values by age

data.sort_values(by = 'age', inplace = True)

In [ ]:
# using plotnine
ggplot(data, aes(x="price")) + geom_histogram(
    aes(y="(stat(count))/sum(stat(count))"),
    binwidth=1000,
    boundary=0,
    color="white",
    fill=color[0],
    size=0.25,
    alpha=0.8,
    show_legend=False,
    na_rm=True,
) + coord_cartesian(xlim=(0, 20000)) + labs(
    x="Price (US dollars)", y="Percent"
) + theme_bw() + expand_limits(
    x=0.01, y=0.01
) + scale_y_continuous(
    expand=(0.01, 0.01), labels=percent_format()
) + scale_x_continuous(
    expand=(0.01, 0.01), breaks=seq(0, 20000, 2500)
)


In [ ]:
# using Pandas plot()
# tedious to plot relative frequencies
data.plot(
    kind = 'hist', figsize = (10,6),
    y = 'price', bins = range(0, data.price.max(), 1000),
    xticks = range(0, data.price.max(), 2000),
    rwidth = 0.9, legend = False, 
    xlabel = 'price in USD', title = 'Absolute frequency by prices')
plt.show();

In [ ]:
# relative frequencies with matplotlib
from matplotlib.ticker import PercentFormatter
fig = plt.figure(figsize = (10,6))
ax = fig.add_subplot(111)
ax.hist(data.price,range(0, data.price.max(), 1000), density = True, rwidth = 0.9, color = 'steelblue')
ax.set_xticks(range(0, data.price.max(), 2000))
ax.set_xlabel('price in USD')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=0.001, decimals = 0))
ax.set_title('Relative frequency of car prices')
plt.show()

In [ ]:
ggplot(data, aes(x="lnprice")) + geom_histogram(
    aes(y="(stat(count)) / sum(stat(count))"),
    binwidth=0.2,
    boundary=0,
    color="white",
    fill=color[0],
    size=0.25,
    alpha=0.8,
    show_legend=False,
    na_rm=True,
) + coord_cartesian(xlim=(6, 10)) + labs(
    x="ln(Price, US dollars)", y="Percent"
) + expand_limits(
    x=0.01, y=0.01
) + scale_y_continuous(
    expand=(0.01, 0.01), labels=percent_format()
) + scale_x_continuous(
    expand=(0.01, 0.01), breaks=seq(6, 10, 1)
) + theme_bw()

In [ ]:
# using Pandas plot()
data.plot(
    kind = 'hist', figsize = (10,6),
    y = 'lnprice', 
    bins = 18,
    rwidth = 0.9, legend = False, 
    xlabel = 'log price in USD', title = 'Frequency by log prices')
plt.show();

### Regression analysis - lo(w)ess

We start with *loess* using first `ggplot` then `seaborn`. 

In [ ]:
#lowess with ggplot
ggplot(data, aes(x="age", y="price")) + geom_point(
    color=color[0], size=1, alpha=0.8, show_legend=False, na_rm=True
) + geom_smooth(method="loess", se=False, colour=color[0], size=1, span=0.9) + labs(
    x="Age (years)", y="Price (US dollars)"
) + theme_bw() + expand_limits(
    x=0.01, y=0.01
) + scale_y_continuous(
    expand=(0.01, 0.01), limits=(0, 20000), breaks=seq(0, 20000, 5000)
) + scale_x_continuous(
    expand=(0.01, 0.01), limits=(0, 30), breaks=seq(0, 30, 5)
)


For `seaborn` it’s recommended to use a Jupyter/IPython interface in [matplotlib mode](https://ipython.readthedocs.io/en/stable/interactive/plotting.html) using the `%matplotlib inline` magic command. 

In [ ]:
%matplotlib inline

In [ ]:
sns.regplot(
    data = data,
    x = 'age', y = 'price', 
    marker= '.',
    fit_reg= True, lowess= True);

In [ ]:
data.describe().T

### Linear regressions

Tools: on of the most-known tools data scientists use for predictive analysis is `scikit-learn`. Here, however, we use the `statsmodels` library that allows users to explore data, estimate statistical models, and perform statistical tests. `Scikit-learn` is great for building all kinds of predictive machine learning models, including linear regression, but spends little effort on providing insights into the models themselves. That's why we turn to `statsmodels` instead. 

#### Model 0: lowess on age

Note: the result of a lo(w)ess regression depends on the tools used. The values calculated below will be different compared to those seen on the `seaborn` regplot output.

In [ ]:
lowess = sm.nonparametric.lowess
y_hat_lowess = lowess(data.price, data.age)

In [ ]:
y_hat_lowess[0:10]

In [ ]:
y_hat_lowess = [x[1] for x in y_hat_lowess]
y_hat_lowess[0:10]


#### Model 1: Linear regression on age

We are building models by adding more and more explanatory variables. 

In [ ]:
reg1 = smf.ols("price ~ age + agesq", data=data).fit(cov_type="HC0")

In [ ]:
type(reg1)

In [ ]:
print(reg1.get_robustcov_results(cov_type='HC1').summary())

In [ ]:
reg1.bic

#### Model 2: We are expanding the base models by adding new explanatory variables

In [ ]:
reg2 = smf.ols("price ~ age + agesq + odometer", data=data).fit(cov_type="HC0")
reg3 = smf.ols(
    "price ~ age + agesq + odometer + odometersq + LE + cond_excellent + cond_good + dealer",
    data=data,
).fit(cov_type="HC0")
reg4 = smf.ols(
    "price ~ age + agesq + odometer + odometersq + LE + XLE + SE + cond_likenew + cond_excellent + cond_good + cylind6 + dealer",
    data=data,
).fit(cov_type="HC0")
reg5 = smf.ols(
    "price ~ age + agesq + odometer + odometersq + LE * age + XLE * age + SE * age + cond_likenew * age + cond_excellent * age + cond_good * age + cylind6 * age + odometer * age + dealer * age",
    data=data,
).fit(cov_type="HC0")

In [ ]:
models = [reg1, reg2, reg3, reg4, reg5]
robustcov_results=[]

for i, model in enumerate(models):
    result=model.get_robustcov_results(cov_type='HC1').summary()
    robustcov_results.append(result)
    print()
    print(f'Regression: reg{i+1}')
    print(result)

In [ ]:
stargazer.Stargazer([reg1])

In [ ]:
ggplot(data, aes(x="age")) + geom_smooth(
    aes(y="price"),colour=color[0],linetype="dashed", method="loess", se=False, size=1
) + geom_line(aes(y="reg1.predict()"), colour=color[1], size=1) + labs(
    x="Age (years)", y="Price (US dollars)"
) + scale_color_manual(
    name="", values=(color[0], color[1]), labels=("Lowess in age", "Quadratic in age")
) + theme_bw() + scale_x_continuous(
    limits=(0, 30), breaks=seq(0, 30, 5)
) + scale_y_continuous(
    limits=(0, 20000), breaks=seq(0, 20000, 5000)
) + theme(
    legend_position=(20, 20),
    legend_direction="horizontal",
    legend_background=element_blank(),
    legend_box_background=element_rect(color="white"),
)

In [ ]:
plt.plot(data.age, reg1.predict(), color = 'steelblue', linestyle = '-')
plt.plot(data.age, y_hat_lowess, color = 'k', linestyle = "--")
plt.legend(labels = ['regression 1', "statsmodel's lowess"])
plt.title("Regression: model 1 vs statsmodel's lowess");

In [ ]:
bic = [round(x.bic, 2) for x in [reg1,reg2,reg3,reg4,reg5]]
sg = stargazer.Stargazer([reg1,reg2,reg3,reg4,reg5])
sg.add_line('BIC', bic, location=stargazer.LineLocation.FOOTER_BOTTOM)
sg

How to tailor-make `Stargazer` output see [here](https://github.com/StatsReporting/stargazer/blob/master/examples.ipynb). 

#### Model 2: Linear Regression with cross validation

In [ ]:
from sklearn.model_selection import KFold
k = KFold(n_splits=4, shuffle=False, random_state=None)

In [ ]:
### Cross validate OLS with combining sklearn k-fold cross validation and statsmodels ols formula


def cv_reg(formula, data, kfold, robustse=None):
    regression_list = []
    predicts_on_test = []
    rsquared = []
    rmse_list = []

    # Calculating OLS for each fold

    for train_index, test_index in k.split(data):
        # print("TRAIN:", train_index, "TEST:", test_index)
        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]
        if robustse is None:
            model = smf.ols(formula, data=data_train).fit()
        else:
            model = smf.ols(formula, data=data_train).fit(cov_type=robustse)
        regression_list += [model]
        predicts_on_test += [model.predict(data_test)]
        rsquared += [model.rsquared]
        rmse_list += [rmse(data_train[formula.split("~")[0]], model.predict())]

    return {
        "regressions": regression_list,
        "test_predict": predicts_on_test,
        "r2": rsquared,
        "rmse": rmse_list,
    }


def summarize_cv(cvlist, stat="rmse"):
    result = pd.DataFrame(
        {"Model" + str(x + 1): cvlist[x][stat] for x in range(len(cv_list))}
    )
    result["Resample"] = ["Fold" + str(x + 1) for x in range(len(cvlist[0]["rmse"]))]
    result = result.set_index("Resample")
    result = pd.concat([result, pd.DataFrame(result.mean(), columns=["Average"]).T])
    return result

In [ ]:
cv1 = cv_reg("price~age+agesq", data, k, "HC0")
cv2 = cv_reg("price~age+agesq+odometer", data, k, "HC0")
cv3 = cv_reg(
    "price~age+agesq+ odometer + odometersq + LE + cond_excellent + cond_good + dealer",
    data,
    k,
    "HC0",
)
cv4 = cv_reg(
    "price~age+agesq+ odometer + odometersq + LE + XLE + SE + cond_likenew + cond_excellent + cond_good + cylind6 + dealer",
    data,
    k,
    "HC0",
)
cv5 = cv_reg(
    "price~age+agesq + odometer + odometersq + LE*age + XLE*age + SE*age + cond_likenew*age + cond_excellent*age + cond_good*age + cylind6*age + odometer*age + dealer*age",
    data,
    k,
    "HC0",
)
cv_list = [cv1, cv2, cv3, cv4, cv5]

In [ ]:
summarize_cv(cv_list)

### Prediction

In [ ]:
data = data[
    [
        "age",
        "agesq",
        "odometer",
        "odometersq",
        "SE",
        "LE",
        "XLE",
        "cond_likenew",
        "cond_excellent",
        "cond_good",
        "dealer",
        "price",
        "cylind6"
    ]
]

In [ ]:
data.dtypes

In [ ]:
new = pd.DataFrame(pd.Series({
    "age":10,
    "agesq":10**2,
    "odometer":12,
    "odometersq":12**2,
    "SE":0,
    "LE":1,
    "XLE":0,
    "cond_likenew":0,
    "cond_excellent":1,
    "cond_good":0,
    "dealer":0,
    "price":np.nan,
    "cylind6":0
})).T
new

In [ ]:
#turning off scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
reg1.resid.describe()

In [ ]:
p1=reg1.get_prediction(new).summary_frame()
p1

In [ ]:
(reg3.fittedvalues-data.price).describe()

In [ ]:
p2=reg3.get_prediction(new).summary_frame()
p2

In [ ]:
#get model3 rmse
rmse(reg3.fittedvalues,data.price)

In [ ]:
pd.DataFrame(
    {
        " ": ["Predicted", "PI_low(95%)", "PI_high(95%)"],
        "Model1": p1[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model3": p2[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
    }
).set_index(" ")

In [ ]:
# summary of predictions and PI 80% version
p1=reg1.get_prediction(new).summary_frame(alpha=0.2)
p2=reg3.get_prediction(new).summary_frame(alpha=0.2)

pd.DataFrame(
    {
        " ": ["Predicted", "PI_low(80%)", "PI_high(80%)"],
        "Model1": p1[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
        "Model3": p2[["mean", "obs_ci_lower", "obs_ci_upper"]].values.tolist()[0],
    }
).set_index(" ")